# EXAMPLE 5
## Explaining the experiment
In this example, we are investigating the configuration memory of an FPGA with 25,611,008 bits (of which aroun 128k are flip-flops, the rest SRAM cells) organized in 800.344x32 words. A golden file following the manufacturer's rules was uploaded on the FPGA and then irradiated in static mode in an only round. Later, the content was downloaded and compared with the initial file.  

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [ ]:
### Defining the path to LELAPE
push!(LOAD_PATH, "PATH_TO_FOLDER_WITH_LELAPE.jl"); # <-- ADAPT THIS INSTRUCTION TO YOUR COMPUTER!
# If you are a Windows user, remember that subfolders are indicated with \\ or /, NEVER with a simple backslash.

In [ ]:
using DelimitedFiles, LELAPE

## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 32 bits
* Memory size in words: 800344.
* In FPGAs, it seems more likely to succeed the POS operation.
* Tests were static. No information about cycles is necessary.

Ok, let us use this information to set these variables:

In [ ]:
LA = 800344 # Memory size in words
WordWidth = 32 # Selfexplaining.
Operation = "POS" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN, CYCLE.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [ ]:
DATA1 = readdlm("ExampleFPGA01.csv", ',', UInt32, '\n', skipstart=1)

Good!! If you have correctly proceeded, a 124x4 unsigned integer matrix is loaded. Now, let us analyze the DATA1 but we need to define in advance several variables to set the analysis.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ function, which returns the MBUs present in DATA.Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [ ]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA1[:,2], DATA1[:,3], WordWidth)

The following loop will show how many MBUs per number of flipped bits were observed:

In [ ]:
for size = 1: WordWidth
    NMBUs = length(findall(MBUSize.==size))
    NMBUs!=0 ? println("$size-bit MBUs: ", NMBUs) : nothing
end

Interesting! There are some MBUs but... are they real or just the accumulation of SBUs. Let us perform an analysis of the presence of false 2- and 3-bit MBUs.

## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

In [ ]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 20

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C1_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C1_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C1_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C1_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [ ]:
C1_SCY, C1_MCU, C1_TRC, C1_SHF = DetectAnomalies_FullCheck(DATA1, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)


Perhaps these matrices are hard to read since, for efficiency, they were returned in UInt32 format, even the number of occurrences!!! Execute the following instrucction for a better comprehension.

In [ ]:
println("Elements appearing more than expected and passing the Self-Consistency test:\n")
for index in 1:length(C1_SCY[:, 1])
    println("Value: 0x", string(C1_SCY[index, 1], base=16, pad = 6), "  (", Int(C1_SCY[index, 1]), ") --> ", Int(C1_SCY[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA1, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

If you had had success in the previous line, you would only have to do the following to integrate the anomalies in only one set. However,  the other sets (C4_MCU; C4_TRC; C4_SHF) since they did not yield any additional result:

In [ ]:
C1_All = [C1_SCY; C1_MCU; C1_TRC; C1_SHF]

## Grouping bitflips
Now, we have discovered those values relating pairs of pseudoaddresses. Now, let us go to group events in DATA. 

The first step consists in labeling all the pseudoaddresses and grouping their assigned indexes to a matrix containing information for the possible MCUs. It is an intermediate step and is done with the instruction _MCU_Indexes_ with the required and already defined parameters. 

In [ ]:
Labeled_addresses = MCU_Indexes(DATA1, Operation, C1_All[:, 1], UsePseudoAddress, WordWidth)

If you are testing ExampleFPGA01, uYou should have got a 32x6 matrix so there are 32 MCUs, of which at least 1 has 6-multiplicity.  

Now, we will classify addresses with _Classify_Addresses_in_MCU()_, using the previous matrix, _Labeled_addresses_. This function returns a vector of matrices. The first element is a matrix with pseudoaddresses related to 9-bit MCUs, and the second and last one to SBUs.

In [ ]:
Events = Classify_Addresses_in_MCU(DATA1, Labeled_addresses, UsePseudoAddress, WordWidth)

---------------------
Difficult to read, isn't it? The following instruction makes the content more readable:

In [ ]:
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    if NMCUs != 0
        for row = 1:NMCUs
            for bit = 1:length(Events)-k+1
                print("0x", string(Events[k][row, bit], base=16, pad = 6), )

                bit != length(Events)-k+1 ? print(", ") : print("\n")

            end
        end
    end
    println()
end

## Are there false 2-bit MCUs?
The only-SBU Model allows estimating the number of false 2-bit events. We can use an implemented function in LELAPE:

In [ ]:
NF2BIT = NF2BitMCUs(DATA1, LA, Operation, length(C4_All[:,1]), WordWidth, KeepCycles, UsePseudoAddress)
println("We expected $NF2BIT false 2-bit MCUs in this  experiment.")

## First part of the analysis completed
You can find in this folder other actual data got from the same memory. Analyze them and, if you wish, add aditional cells combining the anomalous DV values and try to do the most accurate example.

# Second part 
If you perform analysis on all the DATA sets, you will discover that the following values are anomalously repeated and can use them to classify events even if they were not found in its corresponding round. So, let us repeat the calculations with the whole set and refine the results. The values were:

* 1
* 2
* 3230 
* 3231
* 3232 
* 3233
* 3234
* 3235

First, we will load this values in memory:


In [ ]:
Anomalies = convert.(UInt32, [1; 2; 3230:3234]);

Let us load the CSV again

In [ ]:
DATA1 = readdlm("ExampleFPGA01.csv", ',', UInt32, '\n', skipstart=1);

Also, the parameters to define the analysis:

In [ ]:
LA = 800344 # Memory size in words
WordWidth = 32 # Selfexplaining.
Operation = "POS" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 20;

As the anomalies were predefined, it is not necessary to look for them. Besides, the MBU analysis was done before. Let us make groups of events:

In [ ]:
Labeled_addresses = MCU_Indexes(DATA1, Operation, Anomalies, UsePseudoAddress, WordWidth)

And, instead of the indexes in the DATA set, let us recover the original addresses:

In [ ]:
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    if NMCUs != 0
        for row = 1:NMCUs
            for bit = 1:length(Events)-k+1
                print("0x", string(Events[k][row, bit], base=16, pad = 6), )

                bit != length(Events)-k+1 ? print(", ") : print("\n")

            end
        end
    end
    println()
end

##################
println(); println("-"^50); println()
println("Multiplicity\tOccurrences")
println("------------\t-----------")
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    if NMCUs != 0
        println("$(length(Events)-k+1)\t\t$NMCUs")
    end
end

Thats all! Now, you can change the initial data set and procceed. Sometimes the analysis casts better results, sometimes there is not any difference between analysis. Go ahead and verify!

# A little more information
These data sets were used as the basis for J. C. Fabero et al., _"Single Event Upsets Under 14-MeV Neutrons in a 28-nm SRAM-Based FPGA in Static Mode,"_ IEEE Transactions on Nuclear Science, vol. 67, no. 7, pp. 1461-1469, July 2020, doi: 10.1109/TNS.2020.2977874, freely available for download on https://eprints.ucm.es/id/eprint/59496/
